# Ground truth evaluation

#### Import the necessary packages

In [ ]:
import cv2
###################################################
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage as ndi
import pandas as pd
###################################################
from skimage import data
from skimage import img_as_float, img_as_ubyte
from skimage.measure import label, regionprops, regionprops_table
###################################################
import glob
import time
import os
import sys
from pathlib import Path
###################################################
import sklearn
from sklearn.metrics import accuracy_score, balanced_accuracy_score, precision_score, recall_score
from sklearn.metrics import f1_score, jaccard_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

#### Version control

In [ ]:
from datetime import date 
today = date.today().isoformat()

print(f"Notebook last run in {today}")

In [ ]:
sklearn.__version__ #scikit-learn

### Last edited: 20.06.2024

In [ ]:
today = time.strftime("%d_%m_%Y")
today

## Load images

All the necessary images should be in a folder called "sample_nr" (e.g. 2266).

In [ ]:
sample_nr = "2266"

In [ ]:
original = cv2.imread(f"{sample_nr}/2266.11C_crop_GT.tif", cv2.IMREAD_GRAYSCALE)

In [ ]:
plt.imshow(original, cmap="gray")

In [ ]:
img_true = cv2.imread(f"{sample_nr}/all_labels_{sample_nr}.png", cv2.IMREAD_GRAYSCALE)

In [ ]:
img_test = cv2.imread(f"{sample_nr}/2266.11C_segmented.png", cv2.IMREAD_GRAYSCALE)

In [ ]:
img_above = cv2.imread(f"{sample_nr}/AF_{sample_nr}.png", cv2.IMREAD_GRAYSCALE)
img_focus = cv2.imread(f"{sample_nr}/focus_{sample_nr}.png", cv2.IMREAD_GRAYSCALE)
img_below = cv2.imread(f"{sample_nr}/BF_{sample_nr}.png", cv2.IMREAD_GRAYSCALE)
img_orange = cv2.imread(f"{sample_nr}/FOF_{sample_nr}.png", cv2.IMREAD_GRAYSCALE)
img_blue = cv2.imread(f"{sample_nr}/X_{sample_nr}.png", cv2.IMREAD_GRAYSCALE)

In [ ]:
plt.imshow(img_true)

In [ ]:
plt.imshow(img_test)

## Calculate full-image segmentation metrics

In [ ]:
### Shape of the GT-image (Y,X)
shap = img_true.shape
shap

In [ ]:
true_label=label(img_true>0) ###Ground truth
test_label=label(img_test>0) ###Segmented image

In [ ]:
true_obj = np.max(true_label)
seg_obj = np.max(test_label)
print(true_obj,seg_obj)

In [ ]:
img_true_reshape = img_true.reshape(-1)
img_test_reshape = img_test.reshape(-1)

In [ ]:
CM = confusion_matrix(img_true_reshape,img_test_reshape)

In [ ]:
acc = (CM[1][1]+CM[0][0])/(CM[1][1]+CM[0][0]+CM[0][1]+CM[1][0])
print(f"Accuracy: {acc}")

In [ ]:
bacc = balanced_accuracy_score(img_true_reshape,img_test_reshape)
print(f"Balanced accuracy: {bacc}")

In [ ]:
prec = (CM[1][1])/(CM[1][1]+CM[0][1])
print(f"Precision: {prec}")

In [ ]:
rec = (CM[1][1])/(CM[1][1]+CM[1][0])
print(f"Recall: {rec}")

In [ ]:
f1 = f1_score(img_true_reshape,img_test_reshape,pos_label=255)
print(f"F1-score: {f1}")

## Visualize single objects

#### Ground truth and Segmented image

In [ ]:
true_label=label(img_true>0) ###Ground truth
test_label=label(img_test>0) ###Segmented image

In [ ]:
true_obj = np.max(true_label)
seg_obj = np.max(test_label)
print(true_obj,seg_obj)

In [ ]:
true_props=regionprops(true_label,original)
test_props=regionprops(test_label,original)

In [ ]:
print(len(true_props),len(test_props))

In [ ]:
### Choose an air hydrate number to compare labeled and segmented
hydrate = 13

In [ ]:
base_true=np.zeros(img_true.shape, dtype=np.uint8)
base_test=np.zeros(img_test.shape, dtype=np.uint8)

In [ ]:
coordinates_true = true_props[hydrate]["coords"]
coordinates_test = test_props[hydrate]["coords"]

In [ ]:
for coord in coordinates_true: 
        base_true[coord[0],coord[1]]=255

In [ ]:
for coord in coordinates_test: 
        base_test[coord[0],coord[1]]=255

#### Set bounding box expansion

In [ ]:
bounding_box = 12

In [ ]:
transposed = np.transpose(coordinates_true)

In [ ]:
if (np.min(transposed[0])-(bounding_box+1)) < 0:
    xmin = np.min(transposed[0]) 
else:
    xmin = np.min(transposed[0]) - bounding_box
if (np.max(transposed[0])+(bounding_box+1)) > (shap[0]):
    xmax = np.max(transposed[0]) 
else:
    xmax = np.max(transposed[0]) + bounding_box

In [ ]:
if (np.min(transposed[1])-(bounding_box+1)) < 0:
    ymin = np.min(transposed[1]) 
else:
    ymin = np.min(transposed[1]) - bounding_box
if (np.max(transposed[1])+(bounding_box+1)) > (shap[1]):
    ymax = np.max(transposed[1]) 
else:
    ymax = np.max(transposed[1]) + bounding_box

In [ ]:
print(xmin,xmax,ymin,ymax)

In [ ]:
background = np.zeros(base_true[xmin:xmax,ymin:ymax].shape, dtype=np.uint8)

In [ ]:
r=img_as_ubyte(base_true[xmin:xmax,ymin:ymax])
g=background
b=background

In [ ]:
bgr=cv2.merge((b,g,r))

In [ ]:
new=cv2.cvtColor(original[xmin:xmax,ymin:ymax],cv2.COLOR_GRAY2BGR)

In [ ]:
visual_test = cv2.addWeighted(new,0.6,bgr,0.4,0)

#### Plot

In [ ]:
# create figure 
fig = plt.figure(figsize=(12, 12)) 
  
# setting values to rows and column variables
rows = 1
columns = 4
  
# Adds a subplot at the 1st position 
fig.add_subplot(rows, columns, 1) 
  
# showing image 
plt.imshow(base_true[xmin:xmax,ymin:ymax], cmap="gray")
#plt.axis('off') 
plt.title("Ground truth") 
  
# Adds a subplot at the 2nd position 
fig.add_subplot(rows, columns, 2) 
  
# showing image 
plt.imshow(img_test[xmin:xmax,ymin:ymax],cmap="gray")
plt.axis('off') 
plt.title("Segmentation") 
  
# Adds a subplot at the 3rd position 
fig.add_subplot(rows, columns, 3) 
  
# showing image 
plt.imshow(original[xmin:xmax,ymin:ymax],cmap="gray")
plt.axis('off') 
plt.title("Original")

# Adds a subplot at the 4th position 
fig.add_subplot(rows, columns, 4) 
  
# showing image 
plt.imshow(visual_test)
plt.axis('off') 
plt.title("Hand labeled")

placeholder

placeholder

# Compare each object

### Define functions

In [ ]:
def func(nr_obj,img_true,true_props,img_test,bbox_size,a,true_area,slize,TP,TN,FP,FN):
    for amount in range(nr_obj):
        base_true=np.zeros(img_true.shape, dtype=np.uint8)
        coordinates = true_props[a]["coords"]
        for coord in coordinates: 
            base_true[coord[0],coord[1]]=255
        transposed = np.transpose(coordinates)
        if (np.min(transposed[0])-(bbox_size+1)) < 0:
            xmin = np.min(transposed[0]) 
        else:
            xmin = np.min(transposed[0]) - bbox_size
        if (np.max(transposed[0])+(bbox_size+1)) > (shap[0]):
            xmax = np.max(transposed[0]) 
        else:
            xmax = np.max(transposed[0]) + bbox_size
        if (np.min(transposed[1])-(bbox_size+1)) < 0:
            ymin = np.min(transposed[1]) 
        else:
            ymin = np.min(transposed[1]) - bbox_size
        if (np.max(transposed[1])+(bbox_size+1)) > (shap[1]):
            ymax = np.max(transposed[1]) 
        else:
            ymax = np.max(transposed[1]) + bbox_size
        true_crop = base_true[xmin:xmax,ymin:ymax]
        seg_crop = img_test[xmin:xmax,ymin:ymax]
        true_crop_reshape = true_crop.reshape(-1)
        seg_reshape=seg_crop.reshape(-1)
        CM = confusion_matrix(true_crop_reshape,seg_reshape)
        true_area.append(true_props[a]["area"])
        slize.append(true_props[a]["slice"])
        TP.append(CM[1][1])
        TN.append(CM[0][0])
        FP.append(CM[0][1])
        FN.append(CM[1][0])
        a=a+1

In [ ]:
def stats(true_area,slize,TP,TN,FP,FN,bboxx,r):
    seg_img_data = pd.DataFrame()
    seg_img_data["area"] = true_area
    seg_img_data["slice"] = slize
    seg_img_data["TP_full-image"] = TP
    seg_img_data["TN_full-image"] = TN
    seg_img_data["FP_full-image"] = FP
    seg_img_data["FN_full-image"] = FN
    seg_img_data["bounding_box"] = bboxx
    seg_img_data["full_precision"] = seg_img_data["TP_full-image"] / (seg_img_data["TP_full-image"]+seg_img_data["FP_full-image"])
    seg_img_data["true_size"] = seg_img_data["TP_full-image"] + seg_img_data["FN_full-image"]
    seg_size = []
    for obj in range(len(seg_img_data["TP_full-image"])):
        if seg_img_data["TP_full-image"][r] > 0:
            seg_ah_size = seg_img_data["TP_full-image"][r]+seg_img_data["FP_full-image"][r]
            r = r+1
        else:
            seg_ah_size = 0
            r = r+1
        seg_size.append(seg_ah_size)
    seg_img_data["seg_size"] = seg_size
    seg_img_data["diameter_gt"] = 2 * np.sqrt(seg_img_data["area"]/np.pi)
    seg_img_data["diameter_seg"] = 2 * np.sqrt(seg_img_data["seg_size"]/np.pi)
    seg_img_data["diameter_difference"] = seg_img_data["diameter_seg"] - seg_img_data["diameter_gt"]
    seg_img_data["diameter_difference_percent"] = seg_img_data["diameter_difference"] / seg_img_data["diameter_gt"] * 100
    seg_img_data["IoU"] = seg_img_data["TP_full-image"] / (seg_img_data["TP_full-image"]+seg_img_data["FN_full-image"]+seg_img_data["FP_full-image"])
    seg_img_data.fillna(0, inplace=True)
    return seg_img_data

### Set bounding box expansion

In [ ]:
bboxx = 12

## Full image

In [ ]:
TP=[]
TN=[]
FP=[]
FN=[]
true_area=[]
slize=[]

In [ ]:
st = time.time()

In [ ]:
### Call and execute the earlier defined function
func(len(true_props), img_true, true_props, img_test, bboxx, 0, true_area, slize, TP, TN, FP, FN)

In [ ]:
et = time.time()

In [ ]:
elapsed_time = et - st
print('Execution time:', elapsed_time, 'seconds')

In [ ]:
### Call and execute the earlier defined function
data_full = stats(true_area,slize,TP,TN,FP,FN,bboxx,0)

In [ ]:
data_full[0:3]

In [ ]:
pos = len(data_full.loc[data_full['diameter_difference_percent'] > -100])
pos

#### Save data

In [ ]:
now_II = time.strftime("%H-%M")
now_II

In [ ]:
data_full.to_csv(f"{sample_nr}_data-full_{today}_{now_II}.csv",sep=";")

In [ ]:
data_full["full_precision"].plot(kind="hist",bins=100)
plt.title("Precision scores of each AH in GT- vs. seg-image")
plt.xlabel("Precision score")
plt.ylabel("Objects")

In [ ]:
print(f"AH in GT: {true_obj}, Objects in SEG: {seg_obj}, AH erkannt: {pos/true_obj}({pos}), Artefacts: {(seg_obj-pos)/seg_obj}({seg_obj-pos})")

#### Compare mean diameters

In [ ]:
eq_sphere_diameter_gt = []
eq_sphere_diameter_seg = []
area_gt = []
area_seg = []

In [ ]:
for region in true_props:
    size = region["equivalent_diameter_area"]
    eq_sphere_diameter_gt.append(size)

In [ ]:
for region in test_props:
    size = region["equivalent_diameter_area"]
    eq_sphere_diameter_seg.append(size)

In [ ]:
mean_diameter_gt = np.mean(eq_sphere_diameter_gt)
mean_diameter_seg = np.mean(eq_sphere_diameter_seg)
std_diameter_gt = np.std(eq_sphere_diameter_gt)
std_diameter_seg = np.std(eq_sphere_diameter_seg)

In [ ]:
print(f"Ground truth:{np.mean(eq_sphere_diameter_gt)}, Segmented:{np.mean(eq_sphere_diameter_seg)}")

In [ ]:
print(f"Ground truth:{np.std(eq_sphere_diameter_gt)},Segmented:{np.std(eq_sphere_diameter_seg)}")

## Hydrates in focus

In [ ]:
focus_label=label(img_focus>0) ### focus image
focus_obj = np.max(focus_label)
print(true_obj,focus_obj)

In [ ]:
focus_props=regionprops(focus_label,original)

In [ ]:
TP_focus=[]
TN_focus=[]
FP_focus=[]
FN_focus=[]
area_focus=[]
slize_focus=[]

In [ ]:
func(len(focus_props), img_focus, focus_props, img_test, bboxx, 0, area_focus, slize_focus, 
     TP_focus, TN_focus, FP_focus, FN_focus)

In [ ]:
data_focus = stats(area_focus, slize_focus, TP_focus, TN_focus, FP_focus, FN_focus, bboxx, 0)

In [ ]:
len(data_focus["area"])

In [ ]:
data_focus[0:3]

In [ ]:
pos_focus = len(data_focus.loc[data_focus['diameter_difference_percent'] > -100])
pos_focus

## Above focus

In [ ]:
above_label=label(img_above>0) ### AF image
above_obj = np.max(above_label)
print(true_obj,above_obj)

In [ ]:
above_props=regionprops(above_label,original)

In [ ]:
TP_above=[]
TN_above=[]
FP_above=[]
FN_above=[]
area_above=[]
slize_above=[]

In [ ]:
func(len(above_props), img_above, above_props, img_test, bboxx, 0, area_above, slize_above, 
     TP_above, TN_above, FP_above, FN_above)

In [ ]:
data_above = stats(area_above, slize_above, TP_above, TN_above, FP_above, FN_above, bboxx, 0)

In [ ]:
data_above["area"].count()

In [ ]:
pos_above = len(data_above.loc[data_above['diameter_difference_percent'] > -100])
pos_above

## Below focus

In [ ]:
below_label=label(img_below>0) ### BF image
below_obj = np.max(below_label)
print(true_obj,below_obj)

In [ ]:
below_props=regionprops(below_label,original)

In [ ]:
TP_below=[]
TN_below=[]
FP_below=[]
FN_below=[]
area_below=[]
slize_below=[]

In [ ]:
func(len(below_props), img_below, below_props, img_test, bboxx, 0, area_below, slize_below, 
     TP_below, TN_below, FP_below, FN_below)

In [ ]:
data_below = stats(area_below, slize_below, TP_below, TN_below, FP_below, FN_below, bboxx, 0)

In [ ]:
data_below["area"].count()

In [ ]:
pos_below = len(data_below.loc[data_below['diameter_difference_percent'] > -100])
pos_below

## Far out of focus

In [ ]:
orange_label=label(img_orange>0) ### FoF image
orange_obj = np.max(orange_label)
print(true_obj,orange_obj)

In [ ]:
orange_props=regionprops(orange_label,original)

In [ ]:
TP_orange=[]
TN_orange=[]
FP_orange=[]
FN_orange=[]
area_orange=[]
slize_orange=[]

In [ ]:
func(len(orange_props), img_orange, orange_props, img_test, bboxx, 0, area_orange, slize_orange, 
     TP_orange, TN_orange, FP_orange, FN_orange)

In [ ]:
data_orange = stats(area_orange, slize_orange, TP_orange, TN_orange, FP_orange, FN_orange, bboxx, 0)

In [ ]:
data_orange["area"].count()

In [ ]:
pos_orange = len(data_orange.loc[data_orange['diameter_difference_percent'] > -100])
pos_orange

### X

In [ ]:
blue_label=label(img_blue>0) ### X image
blue_obj = np.max(blue_label)
print(true_obj,blue_obj)

In [ ]:
blue_props=regionprops(blue_label,original)

In [ ]:
TP_blue=[]
TN_blue=[]
FP_blue=[]
FN_blue=[]
area_blue=[]
slize_blue=[]

In [ ]:
func(len(blue_props), img_blue, blue_props, img_test, bboxx, 0, area_blue, slize_blue, 
     TP_blue, TN_blue, FP_blue, FN_blue)

In [ ]:
data_blue = stats(area_blue, slize_blue, TP_blue, TN_blue, FP_blue, FN_blue, bboxx, 0)

In [ ]:
data_blue["area"].count()

In [ ]:
pos_blue = len(data_blue.loc[data_blue['diameter_difference_percent'] > -100])
pos_blue

## Collect metrics

In [ ]:
metrics_data = pd.DataFrame(index=[0])

In [ ]:
metrics_data['Max_objects'] = true_obj
metrics_data['Accuracy'] = acc
metrics_data['Balanced_accuracy'] = bacc
metrics_data['Precision'] = prec
metrics_data['Recall'] = rec
metrics_data['F1-score'] = f1

In [ ]:
metrics_data.head()

## Collect values

In [ ]:
values_data = pd.DataFrame(index=[0])

In [ ]:
data_full_pos = data_full.loc[data_full['diameter_difference_percent'] > -100]
full_iou = data_full.loc[data_full['IoU'] > 0.15]

In [ ]:
values_data['Max_objects'] = true_obj
values_data['D diff % std '] = np.std(data_full_pos["diameter_difference_percent"])
values_data['D diff % std IoU'] = np.std(full_iou["diameter_difference_percent"])
values_data['mean diameter gt'] = mean_diameter_gt
values_data['std diameter gt'] = std_diameter_gt
values_data['mean diameter seg'] = mean_diameter_seg
values_data['std diameter seg'] = std_diameter_seg

In [ ]:
values_data.head()

## Collect counts

In [ ]:
print(f"AH in GT: {true_obj}, Objects in SEG: {seg_obj}, AH erkannt: {pos/true_obj}({pos}), Artefacts: {(seg_obj-pos)/seg_obj}({seg_obj-pos})")

In [ ]:
counts_data = pd.DataFrame(index=[0])

In [ ]:
counts_data['Max_objects'] = true_obj
counts_data['Seg_objects'] = seg_obj
counts_data['Air hydrates'] = pos
counts_data['Artifacts'] = (seg_obj-pos)
counts_data['Focus_max'] = focus_obj
counts_data['Focus_seg'] = pos_focus
counts_data['AF_max'] = above_obj
counts_data['AF_seg'] = pos_above
counts_data['BF_max'] = below_obj
counts_data['BF_seg'] = pos_below
counts_data['FoF_max'] = orange_obj
counts_data['FoF_seg'] = pos_orange
counts_data['X_max'] = blue_obj
counts_data['X_seg'] = pos_blue

In [ ]:
counts_data.head()

## Category data

In [ ]:
data_above["category"] = "AF"
data_focus["category"] = "Focus"
data_below["category"] = "BF"
data_orange["category"] = "FOF"
data_blue["category"] = "X"

In [ ]:
seg_img_data = pd.concat([data_above,data_focus,data_below,data_orange,data_blue], ignore_index=True)

In [ ]:
seg_img_data[0:3]

## Save data

In [ ]:
now = time.strftime("%H-%M")
now

In [ ]:
seg_img_data.to_csv(f"{sample_nr}_data_{today}_{now}.csv",sep=";")
metrics_data.to_csv(f"{sample_nr}_metrics_{today}_{now}.csv",sep=";")
values_data.to_csv(f"{sample_nr}_values_{today}_{now}.csv",sep=";")
counts_data.to_csv(f"{sample_nr}_counts_{today}_{now}.csv",sep=";")

# Finished